# RouterChains

In [1]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.chains.llm import LLMChain

from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [2]:
fis_template = ChatPromptTemplate.from_template(
    """
    Você é um professor de física muito inteligente. Você é ótimo em responder perguntas sobre física de forma concisa e fácil de entender.
    Quando você não sabe a resposta para uma pergunta, você admite que não sabe.

    Aqui está uma pergunta: {input}
    """
)

mat_template = ChatPromptTemplate.from_template(
    """
    Você é um matemático muito bom. Você é ótimo em responder perguntas de matemática. Você é tão bom porque consegue decompor problemas difíceis em suas partes componentes, responder às partes componentes e depois juntá-las para responder à pergunta mais ampla.

    Aqui está uma pergunta: {input}
    """
)

hist_template = ChatPromptTemplate.from_template(
    """
    Você é um historiador muito bom. Você tem um excelente conhecimento e compreensão de pessoas, eventos e contextos de uma variedade de períodos históricos. Você tem a capacidade de pensar, refletir, debater, discutir e avaliar o passado. Você tem respeito pela evidência histórica e a capacidade de usá-la para apoiar suas explicações e julgamentos.

    
    Aqui está uma pergunta: {input}
    """
)

In [3]:
prompt_infos = [
    {
        'name': 'Fisica',
        'description': 'ideal para responder perguntas sobre física',
        'prompt_template': fis_template
    },
    {
        'name': 'Matemática',
        'description': 'ideal para responder perguntas sobre matemática',
        'prompt_template': mat_template
    },
    {
        'name': 'História',
        'description': 'ideal para responder perguntas sobre história',
        'prompt_template': hist_template
    }
]

In [4]:
chat = ChatOpenAI(model='gpt-4o-mini')

In [5]:
chains_destino = {}
for info in prompt_infos:
    chain = LLMChain(llm=chat, prompt=info['prompt_template'], verbose=True)
    chains_destino[info['name']] = chain

C:\Users\francisco.procopio\AppData\Local\Temp\ipykernel_8020\936010572.py:3: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=chat, prompt=info['prompt_template'], verbose=True)


In [6]:
chains_destino

{'Fisica': LLMChain(verbose=True, prompt=ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='\n    Você é um professor de física muito inteligente. Você é ótimo em responder perguntas sobre física de forma concisa e fácil de entender.\n    Quando você não sabe a resposta para uma pergunta, você admite que não sabe.\n\n    Aqui está uma pergunta: {input}\n    '), additional_kwargs={})]), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001961EC5E6F0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001961E90A900>, root_client=<openai.OpenAI object at 0x000001961E453440>, root_async_client=<openai.AsyncOpenAI object at 0x000001961EC5E750>, model_name='gpt-4o-mini', model_kwargs={}, openai_api_key=SecretStr('**********')), output_parser=StrOutputParser(), llm_

In [8]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [13]:
destinos = [
    f'{p['name']}: {p['description']}' for p in prompt_infos
]
destinos_str = '\n'.join(destinos)
print(destinos_str)

Fisica: ideal para responder perguntas sobre física
Matemática: ideal para responder perguntas sobre matemática
História: ideal para responder perguntas sobre história


In [14]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinos_str
)

In [16]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
Fisica: ideal para responder perguntas sobre física
Matemática: ideal para responder perguntas sobre 

In [17]:
router_template = PromptTemplate(
    template = router_template,
    input_variables = ['input'],
    output_parser=RouterOutputParser()
)

router_chain = LLMRouterChain.from_llm(
    chat,
    router_template,
    verbose=True
)

In [24]:
default_prompt = ChatPromptTemplate.from_template('{input}')
default_chain = LLMChain(llm=chat, prompt=default_prompt, verbose=True)

chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=chains_destino,
    default_chain=default_chain,
    verbose=True
)

In [25]:
chain.invoke(
    {
        'input': 'O que é um buraco negro?'
    }
)



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
None: {'input': 'O que é um buraco negro?'}

> Entering new LLMChain chain...
Prompt after formatting:
Human: O que é um buraco negro?

> Finished chain.

> Finished chain.


{'input': 'O que é um buraco negro?',
 'text': 'Um buraco negro é uma região do espaço onde a gravidade é tão intensa que nada, nem mesmo a luz, pode escapar de sua atração. Isso ocorre quando uma quantidade significativa de massa é compactada em um espaço muito pequeno, o que gera um campo gravitacional extremamente forte.\n\nOs buracos negros se formam geralmente a partir do colapso de estrelas massivas no final de suas vidas. Quando essas estrelas esgotam seu combustível nuclear, não conseguem mais suportar a força da gravidade e colapsam sob seu próprio peso. Esse colapso pode resultar em um buraco negro.\n\nExistem três tipos principais de buracos negros:\n\n1. **Buracos negros estelares**: Formados a partir do colapso de estrelas massivas. Eles têm algumas vezes algumas vezes a massa do Sol.\n\n2. **Buracos negros supermassivos**: Encontrados no centro de muitas galáxias, incluindo a Via Láctea. Eles podem ter milhões a bilhões de vezes a massa do Sol e a origem exata ainda é obj

In [22]:
chain.invoke(
    {
        'input': 'O que é uma equação quadrática?'
    }
)



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
None: {'input': 'O que é uma equação quadrática?'}

> Entering new LLMChain chain...
Prompt after formatting:
Human: O que é uma equação quadrática?

> Finished chain.

> Finished chain.


{'input': 'O que é uma equação quadrática?',
 'text': 'Uma equação quadrática é uma equação polinomial de grau dois, que pode ser expressa na forma geral:\n\n\\[ ax^2 + bx + c = 0 \\]\n\nonde:\n- \\( x \\) é a variável,\n- \\( a \\), \\( b \\) e \\( c \\) são coeficientes reais (com \\( a \\neq 0 \\) para que a equação seja realmente quadrática).\n\nAs soluções da equação quadrática podem ser encontradas usando a formula de Bhaskara:\n\n\\[ x = \\frac{-b \\pm \\sqrt{b^2 - 4ac}}{2a} \\]\n\nO discriminante \\( \\Delta = b^2 - 4ac \\) determina o número e o tipo de soluções:\n- Se \\( \\Delta > 0 \\), há duas soluções reais e distintas.\n- Se \\( \\Delta = 0 \\), há uma solução real (ou duas soluções reais idênticas).\n- Se \\( \\Delta < 0 \\), não há soluções reais, apenas soluções complexas.\n\nAs equações quadráticas são amplamente utilizadas em várias áreas da matemática e suas aplicações, incluindo física, economia e engenharia.'}

In [ ]:
chain.invoke(
    {
        'input': '?'
    }
)